# Getting started

In this notebook we cover the basics of Pycellin:
- how to build or load a Pycellin model,
- how Pycellin models cell tracking data,
- how to manipulate, modify and enrich the cell lineages,
- how to export the data to other formats.

In [ ]:
import pycellin

## How to get a Pycellin model

### Building from scratch

It is possible to build a Pycellin model manually, starting with an empty model.

In [ ]:
my_model = pycellin.Model()
print(my_model)

You can then fill up the metadata and build cell lineages by adding cells and links. This is covered in details in the notebook [Creating a model from scratch](./Creating%20a%20model%20from%20scratch.ipynb) (WIP).

### Loading from a Pycellin pickle file

You can load a model from a Pycellin pickle file saved on disk with the `load_from_pickle()` method:

In [ ]:
pycellin_pickle = "../sample_data/FakeTracks.pickle"
my_model = pycellin.Model.load_from_pickle(pycellin_pickle)
print(my_model)

However, please note that while `pickle` is a module of the Python Standard Library, is it not secure. **You should only load data from sources you trust.** Please refer to the [documentation of the `pickle` module](https://docs.python.org/3/library/pickle.html) for more information.

### Loading from external tools

Pycellin can load data from different tracking file formats. It currently supports:
- Cell Tracking Challenge text files
- TrackMate XML files

More tracking file formats will be supported in the future.

#### Cell Tracking Challenge

Tracking data formatted as per the [Cell Tracking Challenge](https://celltrackingchallenge.net/) file format [specifications](https://public.celltrackingchallenge.net/documents/Naming%20and%20file%20content%20conventions.pdf) can be loaded with
the `load_CTC_file()` function:

In [ ]:
ctc_file = "../sample_data/FakeTracks_TMtoCTC.txt"
ctc_model = pycellin.load_CTC_file(ctc_file)
print(ctc_model)

For this format, only track topology is read: no cell segmentation is extracted in the case of associated label images (this might get supported later if several people request it).

#### TrackMate



Data generated with [TrackMate](https://imagej.net/plugins/trackmate/) can be loaded into a Pycellin model thanks to the `load_TrackMate_XML()` function:

In [ ]:
trackmate_xml = "../sample_data/Ecoli_growth_on_agar_pad.xml"
tm_model = pycellin.load_TrackMate_XML(trackmate_xml)
print(tm_model)

All data within the TrackMate XML file is loaded into the Pycellin model: there is no loss of information. Most notably, TrackMate properties are accessible within Pycellin under the same name (e.g. AREA, MEAN_INTENSITY_CH1).

To know more about the specifics of using Pycellin with TrackMate data, please refer to the dedicated notebook: [Working with TrackMate data](./Working%20with%20TrackMate%20data.ipynb).

## Pycellin model structure

In [ ]:
# TODO: this section is really really long and I don't think a beginner
# need all of this information. I should synthetize it and move the long
# version into a dedicated notebook.

In this section, we are using the TrackMate model previously loaded as example.

In [ ]:
print(tm_model)

A Pycellin model consists of 3 different elements that we describe in separate sections below:
- the metadata of the model
- the data, i.e. the lineages
- the declaration of the properties present in the lineages

*simplified scheme of a model*

### Model metadata

A pycellin model can stored model metadata, i.e. information about the model and the data of the model.

In Pycellin, the metadata is stored as a dictionary. It is accessible by calling `model_metadata` on your model:

In [ ]:
for metadata_field in tm_model.model_metadata:
    print(metadata_field)

In [ ]:
print(tm_model.model_metadata["provenance"], tm_model.model_metadata["TrackMate_version"])

Some metadata fields are common to most Pycellin models, like `provenance`, `date` or `space_unit`. Others are specific to the way the model was built. For example, models coming from TrackMate have a `TrackMate_version` field and more (`Log`, `GUIState`...).

Since metadata is stored as a dictionary, it is versatile: you can store whatever information you find relevant. The more information you store, the better it is for traceability. For example, you could describe the different channels of your timelapse, or list some image acquisition parameters:

In [ ]:
tm_model.model_metadata["channel1"] = "segmentation"
tm_model.model_metadata["channel2"] = "ZipA-mCherry"
tm_model.model_metadata["objective"] = "100x oil"

However, be careful not to delete the `space_unit`, `time_unit`, `time_step` and `pixel_size` fields when they exist since they can be needed when computing properties like `division_time`, `cell_displacement`...

You can access these important fields with the following methods:

In [ ]:
pix_size = tm_model.get_pixel_size()
s_unit = tm_model.get_space_unit()
print(f"pixel width = {pix_size['width']} {s_unit}")
print(f"pixel height = {pix_size['height']} {s_unit}")
print(f"pixel depth = {pix_size['depth']} {s_unit}")

timestep = tm_model.get_time_step()
t_unit = tm_model.get_time_unit()
print(f"frame = {timestep} {t_unit}")

### Data

In Pycellin, lineages are modeled as directed acyclic graphs or DAGs for short (same as your family tree if we ignore spouses). It means that division events are allowed; they even are recommended if you want to take full advantage of Pycellin. **However, FUSION EVENTS ARE NOT SUPPORTED.** Fusion events happen when a cell has more than one parent. If you try to use Pycellin on a lineage with fusion events, it may crash or produce incorrect results, especially if you are computing properties related to tracking.

Below is an example of a dataset with fusions. If Pycellin detects fusions when loading data from an external tool, a warning will be raised. It is then up to you to decide if you want to proceed or to correct the fusions. However it is highly recommended to correct them.

In [ ]:
trackmate_xml_fusions = "../sample_data/Ecoli_growth_on_agar_pad_with_fusions.xml"
fusion_model = pycellin.load_TrackMate_XML(trackmate_xml_fusions)

You can manually check if a model contains fusions and what are the cells involved with the `get_fusions()` method:

In [ ]:
fusion_model.get_fusions()

To correct the fusions, you can either go back to clean your data with your software of choice or remove one incoming link per fusion cell directly in Pycellin.

In [ ]:
# TODO: a short explanation on cycle lineages is needed here!! And a nice transition.

Fusions or not, lineages data is stored in a `Data` object, accessible by calling `data` on the model:

In [ ]:
print(tm_model.data)

Two types of lineages exist in Pycellin: cell lineages and cell cycle lineages. Both types of lineages are implemented in Pycellin as [NetworkX](https://networkx.org/) DiGraphs. Each lineage is a connected graph built of nodes linked by edges. You can refer to [NetworkX documentation](https://networkx.org/documentation/stable/tutorial.html) for specifics regarding implementation.

#### Cell lineages

In cell lineages, each node of the lineage graph models a cell at a specific point in time and space. Each edge models the displacement of a cell in time and space.

All cell lineages are stored in `cell_data`, a dictionary where keys are the IDs of the lineages, and values the lineages themselves (instances of the CellLineage class):

In [ ]:
# The dict of all lineages.
tm_model.data.cell_data

You can also get a list of the cell lineages with `get_cell_lineages()`:

In [ ]:
cell_lins = tm_model.get_cell_lineages()
for lin in cell_lins:
    print(lin)

To access a specific cell lineage from its `lineage_ID`, you can either query the cell_data dictionary:

In [ ]:
# The lineage whose lineage_ID is 1.
lin1 = tm_model.data.cell_data[1]
print(lin1)

or use the dedicated method:

In [ ]:
# The lineage whose lineage_ID is 2.
lin2 = tm_model.get_cell_lineage_from_ID(2)
print(lin2)

IDs of lineages (`lineage_ID`) are always integer. Lineage IDs are often positive, but not necessarily. Pycellin convention is to attribute a negative ID to one-cell lineage: minus the ID of the cell itself. That way it is easier to distinguish between one-cell lineage and "true" lineage.

##### Plotting a cell lineage

Pycellin offers basic plotting of lineages thanks to the `plot()` method. Cell lineages are plotted like family trees, with time flowing from top to bottom.

Plotting relies on [Plotly](https://plotly.com/python/), an interactive graphing library. You can zoom in and out or pan on the lineage and adjust the axes. Some data is displayed when hovering on cells. Clicking on the legend elements on the right hide or display them.

In [ ]:
lin1.plot()

It is possible to customize and enrich the plot with more information.  
Below is an example where cells are colored according to their area and where cell area is displayed when the cursor is hovering on cells. In this example, area data was extracted from TrackMate (area is called AREA in TrackMate).

In [ ]:
lin_ID = lin1.graph["lineage_ID"]
lin1.plot(
    title=f"Cell lineage of ID = {lin_ID}",  # title of the plot
    node_colormap_prop="AREA",  # property to use to color the nodes
    node_color_scale="plotly3",  # color scale for the node colors
    node_hover_props=["cell_ID", "AREA"],  # which properties to display
)

See Pycellin `plot()` documentation for more information on the available customization parameters.

##### Accessing data in a cell lineage

Just as lineages are identified by their `lineage_ID`, cells are identified by their `cell_ID`, a positive integer. Links between cells (edges) are identified by a tuple of their incoming cell ID and their outgoing cell ID, in this order: `(source_cell_ID, target_cell_ID)`.

Data can be stored at the cell level, at the link level or at the lineage level depending on the information you want to store. Since lineages are implemented as NetworkX DAGs, you can access any properties (called attributes in NetworkX) like in any other NetworkX graph (cf [NetworkX tutorial](https://networkx.org/documentation/stable/tutorial.html)).

For example, if we take the first cell of the lineage above, we can list all the properties associated to the cell (node) as well as their values:

In [ ]:
cell_ID = 8985
lin1.nodes[8985]

To get a specific property value:

In [ ]:
lin1.nodes[8985]["AREA"]

This is the same for links (edges):

In [ ]:
lin1.edges[8985, 9015]  # link between cells 8985 and 9015

In [ ]:
lin1.edges[8985, 9015]["DISPLACEMENT"]  # property loaded from TrackMate

And for lineages:

In [ ]:
lin1.graph

In [ ]:
lin1.graph["NUMBER_SPOTS"]  # number of cells in the lineage, from TrackMate

Aside from accessing data stored in lineages, Pycellin allows you to retrieve noteworthy lineage elements.

In our lineage plotted above, we can easily identify the cells that are dividing with `get_divisions()`:

In [ ]:
print(lin1.get_divisions())

We can similarly identify the first cell of the lineage (root of the graph) and the last cells (leaves of the graph):

In [ ]:
print(f"First cell: {lin1.get_root()}")
print(f"Last cells: {lin1.get_leaves()}")

We can list all the ancestor cells of a specific cell, in chronological order (from the root of the lineage to the cell of interest):

In [ ]:
cell_ID = 9197
print(lin1.get_ancestors(cell_ID))

Or the descendants, unordered:

In [ ]:
print(lin1.get_descendants(cell_ID))

We can also identify sister cells, i.e. cells that are on the same frame and share the same parent cell:

In [ ]:
cell_ID = 8991
lin1.get_sister_cells(cell_ID)

Or all the cells in a cell cycle, from the cell just after division to the next division, in chronological order:

In [ ]:
lin1.get_cell_cycle(cell_ID)

Similarly, we can list all cell cycles in a lineage with `get_cell_cycles()`:

In [ ]:
lin1.get_cell_cycles()

Finally, we can find if a cell is a root, a leaf or a division with `is_root()`, `is_leaf()` or `is_division()` respectively:

In [ ]:
cell_ID = 9105
print(f"For cell {cell_ID}:")
print(f"  is root? {lin1.is_root(cell_ID)}")
print(f"  is leaf? {lin1.is_leaf(cell_ID)}")
print(f"  is division? {lin1.is_division(cell_ID)}")
print()

cell_ID = 9401
print(f"For cell {cell_ID}:")
print(f"  is root? {lin1.is_root(cell_ID)}")
print(f"  is leaf? {lin1.is_leaf(cell_ID)}")
print(f"  is division? {lin1.is_division(cell_ID)}")

#### Cell cycle lineages

Cell cycle lineages, or cycle lineages for short, are like cell lineages except that a node models a whole cell cycle instead of a cell at a specific time point.

When creating a model or loading data for the first time, there is no cycle lineage data. You don't always need it: it depends on the goals of your exploration / analysis.  
If you need it, you can compute and add it with `add_cycle_data()`:


In [ ]:
tm_model.add_cycle_data()

Similar to cell lineages, all cycle lineages are stored in `cycle_data`, a dictionary where keys are the IDs of the lineages, and values the lineages themselves (instances of the CycleLineage class):

In [ ]:
# The dict of all cycle lineages.
tm_model.data.cycle_data

It is also possible to directly get a list of the cycle lineages:

In [ ]:
cycle_lins = tm_model.get_cycle_lineages()
for lin in cycle_lins:
    print(lin)

To get a specific cycle lineage, identified by its `lineage_ID`:

In [ ]:
# Via the dictionary of cycle lineages.
clin1 = tm_model.data.cycle_data[1]  # lineage_ID is 1
print(clin1)

# Via the method get_cycle_lineage_from_ID.
clin2 = tm_model.get_cycle_lineage_from_ID(2)  # lineage_ID is 2
print(clin2)

Cell lineages and cycle lineages share their `lineage_ID`. Thanks to that, it is easier to work with both cell and cycle lineages in parallel. A cell lineage and a cycle lineage with the same ID are just 2 different views of the same underlying lineage.

In [ ]:
lin_ID = 0
lin0 = tm_model.data.cell_data[lin_ID]
clin0 = tm_model.data.cycle_data[lin_ID]
print(f"Lineage of ID {lin_ID} is made of {len(lin0)} cells and has {len(clin0)} cell cycles.")

Cycle lineages are view only: they cannot be modified. This is because cycle data must stay mapped onto cell data to ensure data consistency.

##### Plotting a cycle lineage

Cycle lineages can be plotted like cell lineages. However, while time still flows from top to bottom in the plot, it doesn't flow homogeneously between the different branches of the graph since cell cycles do not have the same length. Consequently, the y-axis represents the level of the cell cycle (i.e. how many cell cycles before the current cycle) instead of time.

In [ ]:
clin1.plot()

In [ ]:
lin_ID = lin1.graph["lineage_ID"]
clin1.plot(
    title=f"Cycle lineage of ID = {lin_ID}",  # title of the plot
    width=1500,  # width of the plot
    height=600,  # height of the plot
    node_text="cycle_ID",  # property to use as text for the nodes
    node_text_font={"color": "white"},  # style of the nodes text
    node_marker_style={"size": 35},  # style of the nodes markers
    node_colormap_prop="cycle_length",  # property to use to color the nodes
    node_color_scale="plotly3",  # color scale for the node colors
    node_hover_props=["cycle_ID", "cycle_length"],  # properties to display
)

##### Accessing data in a cycle lineage

In cycle lineages, cell cycles (nodes) are identified by their `cycle_ID`. The `cycle_ID` is the `cell_ID` of the last cell of the cell cycle, i.e. the division cell.

Otherwise, cycle lineages can be manipulated like cell lineages.

In [ ]:
cycle_ID = 9022

# All properties stored in the cell cycle (node).
print(clin1.nodes[cycle_ID])
# Accessing a specific property value.
print(clin1.nodes[cycle_ID]["cells"])

For now, there is no properties stored in the cycle lineages links and only one property, the `lineage_ID`, in the lineage itself. But accessing the links and lineages properties dictionary is still possible:

In [ ]:
# Properties for links between cell cycles (edges).
print(clin1.edges[cycle_ID, 9057])

# Properties for the cell cycle lineage (graph).
print(clin1.graph)
print(clin1.graph["lineage_ID"])

Like cell lineages, noteworthy cycle lineage elements can be retrieved. Some methods, like `get_divisions()` or `get_sister_cells()` do not make sense on a cycle lineage and are not defined. Others are used in the same way on cell or cycle lineages.

In [ ]:
cycle_ID = 9158

print(f"First cycle: {clin1.get_root()}")
print(f"Last cycles: {clin1.get_leaves()}")

print(f"Ancestor cycles of cycle {cycle_ID}: {clin1.get_ancestors(cycle_ID)}")
print(f"Descendant cycles of cycle {cycle_ID}: {lin1.get_descendants(cycle_ID)}")

print(f"Cycle is root? {clin1.is_root(cycle_ID)}")
print(f"Cycle is leaf? {clin1.is_leaf(cycle_ID)}")

### Declaration of properties

In Pycellin, a property is any data that can be stored into a lineage and that is related to part or all of the lineage (e.g. shape of a cell, velocity of a cell, number of divisions in a lineage...). The definition of a property (i.e. metadata of the property, like its name or units) is stored into a property object.

Here is an example of a mandatory Pycellin property, the `cell_ID`:

In [ ]:
print(pycellin.cell_ID_Property())

You can see that a property is defined by different fields:

**name**  
The name and identifier of the property. It must be **UNIQUE**: two properties cannot share the same name.

**description**  
A concise description of the property.

**provenance**  
Where does the property come from? For imported properties, Pycellin uses the name of the tool the data was imported from, like `TrackMate` or `CTC`. For Pycellin computed properties, Pycellin uses `Pycellin`.

**prop_type**  
The type of graph element the property applies to, either `node`, `edge`, `lineage`.

**lin_type**    
Either `CellLineage`, `CycleLineage` or just `Lineage` depending on which type of lineage your property is related to.

**dtype**   
The Python type of the property values, e.g. `int`, `bool`...

**unit**    
The unit of the property values, e.g. `µm`, `min`, `cell`...

The definition of the all the properties present in a given model is stored into a `PropsMetadata` object as a dictionary (`props`) whose keys are the name of the properties and the values the properties themselves:

In [ ]:
print(tm_model.props_metadata)
print()
for prop in tm_model.props_metadata.props.values():
    print(f"{prop!r}")

This might seem complicated but usually you don't need to manipulate the property declaration nor its properties dictionary. The properties present (i.e. declared) in a model are accessible via the model itself:

In [ ]:
print(tm_model.get_properties().keys())

A few handy methods allow to get a specific subset of declared properties:

In [ ]:
print(f"Node properties: {tm_model.get_node_properties().keys()}")
print(f"Edge properties: {tm_model.get_edge_properties().keys()}")
print(f"Lineage properties: {tm_model.get_lineage_properties().keys()}\n")

print(f"Cell lineage properties: {tm_model.get_cell_lineage_properties().keys()}")
print(f"Cycle lineage properties: {tm_model.get_cycle_lineage_properties().keys()}")

## Managing properties

*Just the basics here, ref the properties notebooks*  
[Managing properties](./Managing%20properties.ipynb)  
[Custom properties](Custom%20properties.ipynb)

Make a table of properties, mandatory, prop type, lin type
- cell_ID
- frame
- lineage_ID

Like previous section, we are using the TrackMate model as example.

In [ ]:
print(tm_model)

## Modification of the lineages

Depending on what you want to do, you may need to manually modify a cell lineage: to remove fusion events, to correct tracking mistakes... or even to totally delete the lineage. Here we cover all the methods that allows you to modify the lineages topology.

### Adding and removing a lineage

To add a lineage to a model, we can first create a new lineage.

In [ ]:
# Creation of the new and empty lineage.
new_lin = pycellin.CellLineage(lid=4)  # a lineage_ID is mandatory here
# Adding the lineage to the model.
new_lin_ID = tm_model.add_lineage(new_lin)
print(tm_model.get_cell_lineage_from_ID(new_lin_ID))

Or let Pycellin create the empty lineage by itself:

In [ ]:
new_lin_ID = tm_model.add_lineage()
print(tm_model.get_cell_lineage_from_ID(new_lin_ID))

Or just specify a lineage ID:

In [ ]:
new_lin_ID = tm_model.add_lineage(lid=10)
print(tm_model.get_cell_lineage_from_ID(new_lin_ID))

To remove a lineage from a model, just specify the lineage_ID to `remove_lineage()`. The removed lineage is returned in case it is needed later.

In [ ]:
removed_lin = tm_model.remove_lineage(10)
print(removed_lin)

### Adding and removing cells

To add a cell, simply call the `add_cell()` method:

In [ ]:
# Adding a cell with ID 1000 to lineage 1 at frame 22.
tm_model.add_cell(
    lid=1,
    cid=1000,
    frame=22,
)

If no `cell_ID` is given, Pycellin will use the next available `cell_ID` and return the value. If no `frame` is given, Pycellin will automatically put the cell at frame 0.

In [ ]:
# Adding a new cell.
new_cell_ID = tm_model.add_cell(
    lid=1,
)
print(f"New cell ID: {new_cell_ID}")

# Then adding some properties that can't be computed to it.
lin1.nodes[new_cell_ID]["cell_x"] = 1.5
lin1.nodes[new_cell_ID]["cell_y"] = 2.5
lin1.nodes[new_cell_ID]["cell_z"] = 0

Properties values can also be assigned at cell creation with the `prop_values` argument, a dictionary with property names as key and property values as values:

In [ ]:
# Adding a new cell with some property values.
new_cell_ID = tm_model.add_cell(
    lid=1,
    frame=1,
    prop_values={"cell_x": 1, "cell_y": 2, "cell_z": 0},
)
print(lin1.nodes[new_cell_ID])

To remove a cell, the `remove_cell()` method needs the ID of the cell to remove as well as the lineage ID. The properties dictionary of the removed node is returned.

In [ ]:
tm_model.remove_cell(cid=9509, lid=1)

### Adding and removing links

Two cells can be linked with the `add_link()` method. It requires the ID of the 2 cells as well as the ID(s) of the lineage(s) the cells belong to. Like cells, properties values can be passed as a dictionary.

In [ ]:
# Linking two cells of the same lineage.
tm_model.add_link(
    source_cid=9503,
    source_lid=1,
    target_cid=1000,
    target_lid=1,  # can be omitted if the same as source
    prop_values={"link_x": 0.5, "link_y": 0.5, "link_z": 0},  # optional
)

In [ ]:
# Linking two cells from different lineages.
# First we need to add a cell to one of our previously created lineage.
new_cell_ID = tm_model.add_cell(lid=4, frame=1)
print(new_cell_ID)

tm_model.add_link(
    source_cid=8985,
    source_lid=1,
    target_cid=new_cell_ID,
    target_lid=4,
)

In the example above, lineage 4 only had one cell, of ID 0. When linking the cell 0, it was removed for lineage 4 and transferred to the source lineage, lineage 1. Lineage 4 is now empty and could be deleted.

We can visualize the modifications done on lineage 1 by plotting it:

In [ ]:
lin1.plot()

Note that you cannot link 2 cells as you want. First, the link is directed to respect the flow of time. The source cell cannot have a time value (`frame` property) greater than the target cell or a `TimeFlowError` will be raised.

Second, creating a link cannot lead to a fusion event since a cell cannot have more than one parent. If it happens, a `FusionError` will be raised.

To remove a cell, specify the IDs of the cells involved in the link and the lineage ID of the cells to `remove_link()`. Like `remove_cell()`, the method returns the properties dictionary of the removed link.

In [ ]:
# We remove the link we created above.
tm_model.remove_link(source_cid=8985, target_cid=0, lid=1)

### Updating the model

Once you have finished modifying the lineages, you need to update it as seen in the [Managing properties](#managing-properties) section. The update will recompute the properties as needed, as well as the cycle lineages if they were added to the model.

In [ ]:
tm_model.update()

As stated before, cycle lineages are read only: you cannot directly add nor remove cell cycles and links. The modifications must be done on the cell lineages. The update of the model will propagate these changes to the cycle lineages. 

## Export

### Pickled Pycellin model

You can save a model on disk anytime with the `save_to_pickle()` method:

In [ ]:
my_model.save_to_pickle("../sample_data/results/FakeTracks_saved.pickle")

A Pycellin model is a complex Python object that can be serialized. Pickling a model is a lossless way to save the model on disk for later use.

However, as stated in [Loading from a Pycellin pickle file](#Loading-from-a-Pycellin-pickle-file), `pickle` module is not secure. Malicious code could be executed when unpickling a file from an unknown source. Because of this safety issue, pickle is not the preferred format for sharing a model with the community.

Indeed, the intended use of `save_to_pickle()` and `load_from_pickle()` is to allow you to save your model whenever you want and to be able to resume working on it at a later time or in another Python session. 

### Tables: DataFrames and CSVs

Lineage data from a Pycellin model can be exported into [Pandas](https://pandas.pydata.org/) [DataFrames](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) and saved as comma-separated values (CSV) files.  

Each type of lineage elements generates a different kind of table, as reviewed below.

#### Cells table

A cell table contains all the properties stored in the **cells (nodes) of the cell lineages** of the model. There is one row per cell and one column per cell property. The DataFrame is ordered by increasing `lineage_ID` first, then by `frame`, then by `cell_ID`.

In [ ]:
cell_df = tm_model.to_cell_dataframe()
print(cell_df.shape)
print(cell_df.columns)

In [ ]:
cell_df.head()

You can then process the DataFrame however you want and save it as a CSV file.

In [ ]:
filename = tm_model.model_metadata["name"] + "_cell_df.csv"
cell_df.to_csv("../sample_data/results/" + filename, index=False)

#### Links table

A link table contains all the properties stored in the **links (edges) of the cell lineages** of the model. There is one row per link and one column per link property. The DataFrame is ordered by increasing `lineage_ID`.

In [ ]:
link_df = tm_model.to_link_dataframe()
print(link_df.shape)
print(link_df.columns)

In [ ]:
link_df.head()

In [ ]:
filename = tm_model.model_metadata["name"] + "_link_df.csv"
link_df.to_csv("../sample_data/results/" + filename, index=False)

#### Cell cycles table

A cell cycle table contains all the properties stored in the **cell cycles (nodes) of the cycle lineages** of the model. There is one row per cell cycle and one column per cell cycle property. The DataFrame is ordered by increasing `lineage_ID` first, then by `level`, then by `cycle_ID`.

In [ ]:
cycle_df = tm_model.to_cycle_dataframe()
print(cycle_df.shape)
print(cycle_df.columns)

In [ ]:
cycle_df.head()

In [ ]:
filename = tm_model.model_metadata["name"] + "_cycle_df.csv"
cycle_df.to_csv("../sample_data/results/" + filename, index=False)

#### Lineages table

A lineage table contains all the properties stored in the **cell lineages (graphs)** of the model. There is one row per lineage and one column per lineage property. The DataFrame is ordered by increasing `lineage_ID`.

In [ ]:
lin_df = tm_model.to_lineage_dataframe()
print(lin_df.shape)
print(lin_df.columns)

In [ ]:
lin_df.head()

In [ ]:
filename = tm_model.model_metadata["name"] + "_lin_df.csv"
lin_df.to_csv("../sample_data/results/" + filename, index=False)

#### Exporting a subset of lineages

For each of the 4 methods above, `to_xxx_dataframe()`, all lineages in the model are processed. If you are interested in just a subset of lineages, you can choose the lineages by passing a list of the `lineage_ID`:

In [ ]:
# Cell dataframe of lineage of ID = 0.
lin0_cell_df = tm_model.to_cell_dataframe([0])
print(lin0_cell_df.shape)
lin0_cell_df.head()

In [ ]:
# Lineage dataframe of lineages of ID 0 and 2.
subset_lin_df = tm_model.to_lineage_dataframe([0, 2])
print(subset_lin_df.shape)
subset_lin_df.head()

### Tracking formats

Pycellin can export data to different external tracking file formats. It currently supports:
- Cell Tracking Challenge text files
- TrackMate XML files

More tracking file formats will be supported in the future.

#### Cell Tracking Challenge

Simply use the `export_CTC_file()` function to export a model to a CTC compatible text file:

In [ ]:
filename = my_model.model_metadata["name"] + "_CTC_export.txt"
ctc_out = "../sample_data/results/" + filename
pycellin.export_CTC_file(my_model, ctc_out)

The CTC file format does not support properties: only the topology of the lineages is exported. Moreover, `lineage_ID` and `cell_ID` are not carried over.

See the [CTC file format specifications](https://public.celltrackingchallenge.net/documents/Naming%20and%20file%20content%20conventions.pdf) if you want more information on the format.

#### TrackMate

A Pycellin model is exported as a TrackMate XML file with the function `export_TrackMate_XML()`. However, TrackMate requires properties to be given in a unique temporal and a unique spatial unit while Pycellin properties can each have different units. So you need to specify the space and time units for TrackMate and to ensure that your units are consistent across the Pycellin properties of the model.

In [ ]:
filename = my_model.model_metadata["name"] + "_TrackMate_export.xml"
xml_out = "../sample_data/results/" + filename
pycellin.export_TrackMate_XML(my_model, xml_out, {"spatialunits": "pixel", "temporalunits": "sec"})

Properties added to the model with Pycellin will be carried over to TrackMate if the data type of the property is integer, float or boolean. `lineage_ID` and `cell_ID` are converted into their TrackMate equivalent: `TRACK_ID` and `SPOT_ID` respectively.

For a more detailed explanation, please refer to the dedicated notebook: [Working with TrackMate data](./Working%20with%20TrackMate%20data.ipynb).